In [ ]:
pip install ipywidgets

In [1]:
#Importing libraries

import torch
from torchvision.datasets import CocoDetection
from torchvision import transforms
from yolov5.models.yolo import Model
import matplotlib.pyplot as plt

In [3]:
# Loading the pre-trained YOLOv5 model

model = torch.hub.load('ultralytics/yolov5', 'yolov5s')
model

Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2023-3-30 Python-3.8.10 torch-1.14.0a0+44dac51 CUDA:0 (NVIDIA GeForce RTX 3070 Laptop GPU, 7949MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


AutoShape(
  (model): DetectMultiBackend(
    (model): DetectionModel(
      (model): Sequential(
        (0): Conv(
          (conv): Conv2d(3, 32, kernel_size=(6, 6), stride=(2, 2), padding=(2, 2))
          (act): SiLU(inplace=True)
        )
        (1): Conv(
          (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
          (act): SiLU(inplace=True)
        )
        (2): C3(
          (cv1): Conv(
            (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv2): Conv(
            (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv3): Conv(
            (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (m): Sequential(
            (0): Bottleneck(
              (cv1): Conv(
                (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1))
  

In [5]:
data_dir = "dataset"

In [6]:
# Loading the COCO dataset

coco = CocoDetection(root='dataset', annFile=f"{data_dir}/instances_val2017.json", transform=transforms.ToTensor())


NOTE! Installing ujson may make loading annotations faster.
Loading annotations into memory...
Done (t=0.38s)
Creating index...
index created!


In [8]:
# Selecting an image from the dataset

image, target = coco[0]
image

tensor([[[0.66667, 0.67843, 0.68627,  ..., 0.27059, 0.26667, 0.27451],
         [0.67451, 0.69020, 0.69412,  ..., 0.27059, 0.28235, 0.27843],
         [0.68627, 0.69412, 0.69804,  ..., 0.27451, 0.27059, 0.27843],
         ...,
         [0.73725, 0.71765, 0.75686,  ..., 0.72941, 0.72941, 0.73333],
         [0.72941, 0.73333, 0.72941,  ..., 0.77647, 0.76471, 0.72941],
         [0.72941, 0.73333, 0.72941,  ..., 0.50588, 0.49412, 0.41961]],

        [[0.53333, 0.55686, 0.56471,  ..., 0.29804, 0.29804, 0.27843],
         [0.55294, 0.56863, 0.57255,  ..., 0.30196, 0.31373, 0.29412],
         [0.56471, 0.57255, 0.57647,  ..., 0.30588, 0.30196, 0.29412],
         ...,
         [0.74118, 0.71765, 0.73333,  ..., 0.61569, 0.61569, 0.61176],
         [0.71765, 0.72157, 0.71765,  ..., 0.52549, 0.47059, 0.34510],
         [0.71765, 0.72157, 0.71765,  ..., 0.23529, 0.22353, 0.16078]],

        [[0.28627, 0.30196, 0.30980,  ..., 0.16471, 0.15294, 0.14510],
         [0.30196, 0.31373, 0.31765,  ..., 0.

In [10]:
# Generating a square-shaped adversarial patch

patch_size = 50
patch = torch.rand((patch_size, patch_size, 3), requires_grad=True)

In [11]:
# Overlaying the patch on the image to create the adversarial input image

x1 = 100  # top-left x-coordinate of the patch
y1 = 100  # top-left y-coordinate of the patch
x2 = x1 + patch_size  # bottom-right x-coordinate of the patch
y2 = y1 + patch_size  # bottom-right y-coordinate of the patch
adversarial_image = image.clone()
patch_location = torch.zeros_like(adversarial_image)
patch_location[y1:y2, x1:x2, :] = 1
patched_patch = patch.repeat(patch_size, patch_size, 10, 25) * patch_location
adversarial_image += patched_patch[0]


RuntimeError: The size of tensor a (75) must match the size of tensor b (640) at non-singleton dimension 3

In [12]:
# Passing the adversarial input image through the YOLOv5 model

output = model(adversarial_image.unsqueeze(0))

RuntimeError: Sizes of tensors must match except in dimension 1. Expected size 28 but got size 27 for tensor number 1 in the list.

In [13]:
# Displaying the original image, the adversarial image, and the model's prediction

fig, axs = plt.subplots(1, 3, figsize=(15, 5))
axs[0].imshow(image.permute(1, 2, 0))
axs[0].set_title('Original Image')
axs[1].imshow(adversarial_image.permute(1, 2, 0))
axs[1].set_title('Adversarial Image')
axs[2].imshow(image.permute(1, 2, 0))
axs[2].set_title('Prediction: {}'.format(output))
plt.show()

NameError: name 'output' is not defined